In [41]:
import cv2
import re
import json
import pytesseract
import argparse
import editdistance

In [23]:
cap = cv2.VideoCapture('./Meditation Questions and Answers, April 20, 2020-Ocb8b9fIBnc.mkv')

In [24]:
fps = cap.get(cv2.CAP_PROP_FPS) # https://stackoverflow.com/a/52032374
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = frame_count/fps

In [25]:
def get_text_at_second(t):
    cap.set(cv2.CAP_PROP_POS_MSEC, 1000 * t)
    ret, frame = cap.read()
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    return pytesseract.image_to_string(img_rgb)

In [29]:
def is_same_noisy_text(a, b):
    return editdistance.distance(a.lower(), b.lower()) < 3

In [31]:
%%time
t = 0

current_text = ''
new_text = ''


texts = []

current_text = ''
current_text_start_pos = -1

for t in range(0, int(duration), 30):
    text = get_text_at_second(t)
    
    if is_same_noisy_text(text, current_text):
        # same 'slide'
        continue
        
    if current_text_start_pos == -1:
        texts.append((t, text))
        current_text = text
        current_text_start_pos = t
    else:
        found = False
        
        for j in range(t, current_text_start_pos, -1):
            temp_text = get_text_at_second(j)
            
            if is_same_noisy_text(temp_text, text):
                current_text = text
                current_text_start_pos = j
                texts.append((j, text))
                found = True
                break
        
        if not found:
            print('ERROR: NOT FOUND', t, text)
            break

CPU times: user 4min 26s, sys: 7.09 s, total: 4min 33s
Wall time: 3min 28s


In [34]:
texts

[(0, 'Welcome\n\nSirimangalo.org'),
 (270,
  'Question\n\nare there any mediatations on suicide?\n\nsirimangalo.org'),
 (390,
  'Question\n\nCan meditation take all the pain away?\n\nsirimangalo.org'),
 (510,
  'Question\n\nWhat to do when no thoughts arise in meditation?\nShould | just stay with the rising and falling and not\nworry myself with a lack of thought?\n\nsirimangalo.org'),
 (630,
  'Question\n\nWhen attachments are too strong, how can we resist the\ninsecurity and let go? | feel constantly that | need to\nhold on, even when things don’t serve. Letting go\nmakes me feel vulnerable\n\nsirimangalo.org'),
 (660,
  "Question\n\nhow do you meditate when you're anxious and finding it\nhard to relax?\n\nsirimangalo.org"),
 (750,
  'Question\n\nApart from the precepts what besides meditation must\nwe do?\n\nsirimangalo.org'),
 (810,
  'Question\n\nApart from the precepts what besides meditation must\nVimo loys\n\nsirimangalo.org'),
 (870,
  'Question\n\nApart from the precepts what

In [52]:
def clean_text(text):
    cleaned = text
    cleaned = text.replace('\n', ' ')
    cleaned = re.sub(r'\s+', ' ', cleaned)
    cleaned = cleaned.strip()
    
    cleaned = re.sub(r'^Question', '', cleaned, re.I)
    cleaned = re.sub(r'sirimangalo\.org$', '', cleaned, re.I)
    
    cleaned = cleaned.replace('|', 'I')
    
    return cleaned.strip()

In [ ]:
{
  "session": {
    "mediaUrl": "https://www.youtube.com/watch?v=Ocb8b9fIBnc",
    "tags": [
      "Dhamma",
      "Yuttadhammo Bhikkhu"
    ],
    "name": "The Veil of Ignorance",
    "_id": "session-1592158372006"
  },
  "notes": [
    {
      "text": "fsdfsdfsdf",
      "time": 880,
      "_id": "note-session-1592158372006-1593265745012"
    }
  ]
}

In [53]:
for t, text in texts:
    print(t, clean_text(text))

0 Welcome Sirimangalo.org
270 are there any mediatations on suicide?
390 Can meditation take all the pain away?
510 What to do when no thoughts arise in meditation? Should I just stay with the rising and falling and not worry myself with a lack of thought?
630 When attachments are too strong, how can we resist the insecurity and let go? I feel constantly that I need to hold on, even when things don’t serve. Letting go makes me feel vulnerable
660 how do you meditate when you're anxious and finding it hard to relax?
750 Apart from the precepts what besides meditation must we do?
810 Apart from the precepts what besides meditation must Vimo loys
870 Apart from the precepts what besides meditation must we do?
900 Is it a good idea to change your environment while meditating?
960 The past is one of my greatest attachments and I struggle to let go of it. Any other tips of letting go of the past in addition to saying, "thinking, thinking" when it arises?
1020 i have schizophrenia and meditat